<a href="https://colab.research.google.com/github/DanielWarfield1/MLWritingAndResearch/blob/main/CUDA_cheet_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA cheet sheet
Runtime must be NVIDIA GPU

Based on these amazing tutorials:

https://www.youtube.com/watch?v=m0nhePeHwFs&list=PLKK11Ligqititws0ZOoGk3SW-TZCar4dK&index=2

# Setting up CUDA

In [1]:
!pip install nvcc4jupyter

In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpd3e4ofbt".


# Hello World in CUDA

In [3]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

__host__ int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1



In [4]:
%%cuda
#include <stdio.h>

__global__ void hello() {
    printf("Hello from block: (%u, %u, %u), thread: (%u, %u, %u)\n",
           blockIdx.x, blockIdx.y, blockIdx.z,
           threadIdx.x, threadIdx.y, threadIdx.z);
}

int main() {
    // Define the dimensions of the grid and blocks
    dim3 gridDim(2, 2, 2);   // 2x2x2 grid of blocks
    dim3 blockDim(2, 2, 2);  // 2x2x2 grid of threads per block

    // Launch the kernel
    hello<<<gridDim, blockDim>>>();

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    return 0;
}

Hello from block: (0, 0, 1), thread: (0, 0, 0)
Hello from block: (0, 0, 1), thread: (1, 0, 0)
Hello from block: (0, 0, 1), thread: (0, 1, 0)
Hello from block: (0, 0, 1), thread: (1, 1, 0)
Hello from block: (0, 0, 1), thread: (0, 0, 1)
Hello from block: (0, 0, 1), thread: (1, 0, 1)
Hello from block: (0, 0, 1), thread: (0, 1, 1)
Hello from block: (0, 0, 1), thread: (1, 1, 1)
Hello from block: (0, 0, 0), thread: (0, 0, 0)
Hello from block: (0, 0, 0), thread: (1, 0, 0)
Hello from block: (0, 0, 0), thread: (0, 1, 0)
Hello from block: (0, 0, 0), thread: (1, 1, 0)
Hello from block: (0, 0, 0), thread: (0, 0, 1)
Hello from block: (0, 0, 0), thread: (1, 0, 1)
Hello from block: (0, 0, 0), thread: (0, 1, 1)
Hello from block: (0, 0, 0), thread: (1, 1, 1)
Hello from block: (1, 0, 1), thread: (0, 0, 0)
Hello from block: (1, 0, 1), thread: (1, 0, 0)
Hello from block: (1, 0, 1), thread: (0, 1, 0)
Hello from block: (1, 0, 1), thread: (1, 1, 0)
Hello from block: (1, 0, 1), thread: (0, 0, 1)
Hello from bl

# First Kernel
https://www.youtube.com/watch?v=rbHURDt2dcQ&list=PLKK11Ligqititws0ZOoGk3SW-TZCar4dK&index=2

---

a kernel is a function that does something on a GPU.

---

`__global__` is a kernel. It's called by the host (cpu) but it runs on the device (gpu)

---

`cudaMalloc` is a function that reserves global memory on the GPU. It takes two arguments:
- a pointer on the device (GPU)
- some ammount of space on the GPU in bytes.

---

`cudaFree` frees up memory on the device. Accepts a pointer

---

`cudaMemcpy` copies data from the host (computer RAM) to the device (GPU memory) and vice versa. Accepts four arguments
- a pointer for the destination
- a pointer for the source
- a size in bytes
- a direction (for instance, from the device to the host, or from the host to device).

Keep in mind, this is highly efficient C++. That means we don't get a lot of abstractions to help us out.

---

First try, getting GPU to add two numbers together:
- Create two ints
- allocate mem on device
- copy values
- call a kernel to add them
- copy back
- print out results
- free device memory

In [5]:
%%cuda
#include <iostream>
#include <cuda.h>

using namespace std;

// Defining the kernel
__global__ void addIntsCUDA(int *a, int *b) {
    a[0] += b[0];
}

// Running main on host, which triggers the kernel
int main() {
    // Host values
    int a = 1, b = 2;

    //printing expression
    cout << a << " + " << b <<" = ";

    // Device pointers (GPU)
    int *d_a, *d_b;

    // Allocating memory on the device (GPU)
    cudaMalloc(&d_a, sizeof(int));
    cudaMalloc(&d_b, sizeof(int));

    // Copying values from the host (CPU RAM) to the device (GPU)
    cudaMemcpy(d_a, &a, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, &b, sizeof(int), cudaMemcpyHostToDevice);

    // Calling the kernel to add the two values at the two pointer locations.
    addIntsCUDA<<<1, 1>>>(d_a, d_b);

    // The addition function overwrites the a pointer with the sum. Thus
    // this copies the result.
    cudaMemcpy(&a, d_a, sizeof(int), cudaMemcpyDeviceToHost);

    //printing result
    cout << a << endl;

    //freeing memory.
    cudaFree(d_a);
    cudaFree(d_b);

    return 0;
}

1 + 2 = 3



# Multi Threaded Addition
https://www.youtube.com/watch?v=kzXjRFL-gjo&list=PLKK11Ligqititws0ZOoGk3SW-TZCar4dK&index=4

---

`thead` is a single execution unit that runs a kernel.

---

`thread block` all the threads in a block can communicate with one another.

---

`grid` is a bunch of thread blocks.

---

`dim3` a 3d structure in CUDA
not super clear exactly what this is in the video. Will be elaborated more

---

`launch configuration` something like `<<<100,256>>>` would launch 100 blocks of 256 threads. One invokes a kernel with a launch configuration. Like `myKernel<<<100, 256>>>(arguments)`. It seems like you can call 1024 threads per block, and you can call a lot of blocks (like 2^31-1)

There's a limit on the number of threads per block, but an almost infinite number of blocks. A big GPU might run multiple blocks at once, a weaker GPU might only run one or two blocks at once. This makes high powered GPUs run similarly to low powered GPUs, just faster.

---

each thread is an individula with some unique identifiers that allow the threads to do certain things

- threadIdx
- blockIdx
- blockDim
- gridDim

these allow a thread to get an idea of which thread it is within a block, and which block it exists in within a set of blocks.

---

We can use this to parallelize a whole bunch of addition.




In [6]:
%%cuda
#include <iostream>
#include <cuda.h>
#include <stdlib.h>
#include <ctime>
#include <chrono>

using namespace std::chrono;
using namespace std;

__global__ void addInts(int *a, int *b, int count){
    int id = blockIdx.x* blockDim.x + threadIdx.x;
    if(id < count){
        a[id] += b[id];
    }
}

int main(){
    srand(time(NULL));
    int count = 500000000;
    int *h_a = new int[count];
    int *h_b = new int[count];

    for (int i = 0; i < count; i++){
        h_a[i] = rand() % 1000;
        h_b[i] = rand() % 1000;
    }

    auto start = high_resolution_clock::now();

    cout<<"First 5 Calculations out of "<<count<<" : "<<endl;
    for(int i = 0; i < 5; i++){
        cout<<h_a[i]<<" + "<<h_b[i]<<endl;
    }

    int *d_a, *d_b;

    if(cudaMalloc(&d_a, sizeof(int) * count) != cudaSuccess){
        cout<<"error allocating a";
        return 0;
    }

    if(cudaMalloc(&d_b, sizeof(int) * count) != cudaSuccess){
        cout<<"error allocating b";
        cudaFree(d_a);
        delete[] h_a;
        delete[] h_b;
        return 0;
    }

    if(cudaMemcpy(d_a, h_a, sizeof(int) * count, cudaMemcpyHostToDevice) != cudaSuccess){
        cout<<"could not copy"<<endl;
        cudaFree(d_a);
        cudaFree(d_b);
        delete[] h_a;
        delete[] h_b;
        return 0;
    }

    if(cudaMemcpy(d_b, h_b, sizeof(int) * count, cudaMemcpyHostToDevice) != cudaSuccess){
        cout<<"could not copy"<<endl;
        cudaFree(d_a);
        cudaFree(d_b);
        delete[] h_a;
        delete[] h_b;
        return 0;
    }

    auto calcStart = high_resolution_clock::now();
    addInts<<<count/1024+1, 1024>>>(d_a, d_b, count);
    auto calcStop = high_resolution_clock::now();

    if(cudaMemcpy(h_a, d_a, sizeof(int)*count, cudaMemcpyDeviceToHost) !=cudaSuccess){
        cudaFree(d_a);
        cudaFree(d_b);
        delete[] h_a;
        delete[] h_b;
        cout<<"error copying from device"<<endl;
        return 0;
    }

    auto stop = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(stop - start);

    cout<<endl<<"Results from GPU:"<<endl;
    for(int i = 0; i < 5; i++){
        cout<<h_a[i]<<endl;
    }

    cout << endl << "time to do everything: " << duration.count() << " microseconds" << endl;
    auto calcDuration = duration_cast<microseconds>(calcStop - calcStart);
    cout << endl << "time to do only calculation: " << calcDuration.count() << " microseconds" << endl;

    cudaFree(d_a);
    cudaFree(d_b);
    delete[] h_a;
    delete[] h_b;

    return 0;
}

First 5 Calculations out of 500000000 : 
841 + 299
944 + 447
395 + 755
952 + 235
367 + 421

Results from GPU:
1140
1391
1150
1187
788

time to do everything: 1427178 microseconds

time to do only calculation: 39719 microseconds



## Comparing to CPU

In [7]:
%%cuda
#include <iostream>
#include <thread>
#include <vector>
#include <stdlib.h>
#include <ctime>
#include <chrono>

using namespace std::chrono;
using namespace std;

void addInts(int *a, int *b, int start, int end) {
    for (int i = start; i < end; ++i) {
        a[i] += b[i];
    }
}

int main() {
    srand(time(NULL));
    int count = 500000000;
    int *h_a = new int[count];
    int *h_b = new int[count];

    for (int i = 0; i < count; i++) {
        h_a[i] = rand() % 1000;
        h_b[i] = rand() % 1000;
    }

    auto start = high_resolution_clock::now();

    cout << "First 5 Calculations out of " << count << " : " << endl;
    for (int i = 0; i < 5; i++) {
        cout << h_a[i] << " + " << h_b[i] << endl;
    }

    int numThreads = 8;
    vector<thread> threads;
    int chunkSize = count / numThreads;

    for (int i = 0; i < numThreads; ++i) {
        int startIdx = i * chunkSize;
        int endIdx = (i == numThreads - 1) ? count : startIdx + chunkSize;
        threads.push_back(thread(addInts, h_a, h_b, startIdx, endIdx));
    }

    for (auto &t : threads) {
        t.join();
    }

    auto stop = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(stop - start);

    cout << endl << "Results from CPU:" << endl;
    for (int i = 0; i < 5; i++) {
        cout << h_a[i] << endl;
    }

    cout << endl << "time to do all calculations: " << duration.count() << " microseconds" << endl;

    delete[] h_a;
    delete[] h_b;

    return 0;
}

First 5 Calculations out of 500000000 : 
638 + 932
663 + 716
385 + 660
874 + 113
124 + 716

Results from CPU:
1570
1379
1045
987
840

time to do all calculations: 310649 microseconds



example of how ludicrously long it takes a GPU to get data vs how fast the calculations get done.

## Memory in CUDA
https://www.youtube.com/watch?v=RY2_8wB2QY4&list=PLKK11Ligqititws0ZOoGk3SW-TZCar4dK&index=5

---

**note this is an old video, so some of this might be pretty different**

Host can read or write to:
- Global
- Constant
- Texture

All blocks can read or write from Global, constant (?) or texture (?)

Blocks have shared memory that's accessible to each thread, each thread has local memory and registers.

Global memory:
- Global is big and slow
- cudaMalloc, cudaFree, cudaMemcpy, cudaMemset all deal with global memory
- This is the main memory of the graphics card.
- GPU and CPU can address every byte
- persistant across kernel calls

Local Memory:
- also slow
- used when registers can't be used, or when we run out
- used for arrays and big objects
- Cached (L1 and L2 cache), so the compiler tries it's best to manage this memory to preserve performance.

Caches:
- L1 cache is really fast (basically a partition of shared memory)
- L2 is slower, all shared memory go through L2
- you can turn caching on and off

Constant Memory:
- part of GPU main memory
- very close to threads, so fast
- CPU can write, GPU can only read
- very fast
- not a lot of it (in this old 10 year old tutorial)

Texture Memory:
- Read only by GPU, set up by CPU
- designed to do tecture fetching
- interpolates pixels
- cached in a way that there are quirks

Shared memory
- very fast
- try not to serialize, want all threads to access at the same time.
- an L1 cache that we're in control of

Registers
- basically a variable. int i=5, for instance. Unless we run out, then local memory will be used
- scope is per thread
- Streaming multiprocessors (sms) can execute more than one thread per time, if resources permit (?) so being light on register use is useful

breakdown image at 14:36

In [8]:
%%cuda

#include <iostream>
#include <cuda_runtime.h>

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);

    std::cout << "Number of CUDA devices: " << deviceCount << std::endl;

    for (int device = 0; device < deviceCount; ++device) {
        cudaDeviceProp deviceProp;
        cudaGetDeviceProperties(&deviceProp, device);

        std::cout << "\nDevice " << device << ": \"" << deviceProp.name << "\"" << std::endl;
        std::cout << "  Total amount of global memory: " << deviceProp.totalGlobalMem << " bytes" << std::endl;
        std::cout << "  Total number of multiprocessors: " << deviceProp.multiProcessorCount << std::endl;
        std::cout << "  Total amount of constant memory: " << deviceProp.totalConstMem << " bytes" << std::endl;
        std::cout << "  Total amount of shared memory per block: " << deviceProp.sharedMemPerBlock << " bytes" << std::endl;
        std::cout << "  Total number of registers available per block: " << deviceProp.regsPerBlock << std::endl;
        std::cout << "  Warp size: " << deviceProp.warpSize << std::endl;
        std::cout << "  Maximum number of threads per block: " << deviceProp.maxThreadsPerBlock << std::endl;
        std::cout << "  Maximum sizes of each dimension of a block: " << deviceProp.maxThreadsDim[0] << " x " << deviceProp.maxThreadsDim[1] << " x " << deviceProp.maxThreadsDim[2] << std::endl;
        std::cout << "  Maximum sizes of each dimension of a grid: " << deviceProp.maxGridSize[0] << " x " << deviceProp.maxGridSize[1] << " x " << deviceProp.maxGridSize[2] << std::endl;
        std::cout << "  Clock rate: " << deviceProp.clockRate << " kilohertz" << std::endl;
        std::cout << "  Memory clock rate: " << deviceProp.memoryClockRate << " kilohertz" << std::endl;
        std::cout << "  Memory bus width: " << deviceProp.memoryBusWidth << " bits" << std::endl;
    }

    return 0;
}

Number of CUDA devices: 1

Device 0: "Tesla T4"
  Total amount of global memory: 15835660288 bytes
  Total number of multiprocessors: 40
  Total amount of constant memory: 65536 bytes
  Total amount of shared memory per block: 49152 bytes
  Total number of registers available per block: 65536
  Warp size: 32
  Maximum number of threads per block: 1024
  Maximum sizes of each dimension of a block: 1024 x 1024 x 64
  Maximum sizes of each dimension of a grid: 2147483647 x 65535 x 65535
  Clock rate: 1590000 kilohertz
  Memory clock rate: 5001000 kilohertz
  Memory bus width: 256 bits



## Playing with the Nvidia Profiler
- writes some code to a file
- compiles it
- runs the profiler

In [9]:
%%writefile addInts.cu
#include <iostream>
#include <cuda.h>
#include <stdlib.h>
#include <ctime>

using namespace std;

__global__ void addInts(int *a, int *b, int count){
    int id = blockIdx.x* blockDim.x + threadIdx.x;
    if(id < count){
        a[id] += b[id];
    }
}

int main(){
    srand(time(NULL));
    int count = 10000;
    int *h_a = new int[count];
    int *h_b = new int[count];

    for (int i = 0; i < count; i++){
        h_a[i] = rand() % 1000;
        h_b[i] = rand() % 1000;
    }

    int *d_a, *d_b;

    cudaMalloc(&d_a, sizeof(int) * count);
    cudaMalloc(&d_b, sizeof(int) * count);

    cudaMemcpy(d_a, h_a, sizeof(int) * count, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int) * count, cudaMemcpyHostToDevice);

    addInts<<<count/256+1, 256>>>(d_a, d_b, count);

    cudaMemcpy(h_a, d_a, sizeof(int) * count, cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_b);
    delete[] h_a;
    delete[] h_b;

    return 0;
}

Writing addInts.cu


In [10]:
!nvcc addInts.cu
!./a.out

In [11]:
!nvcc addInts.cu -o addInts
!nvprof ./addInts

==2277== NVPROF is profiling process 2277, command: ./addInts
==2277== Profiling application: ./addInts
==2277== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   58.91%  12.160us         2  6.0800us  6.0800us  6.0800us  [CUDA memcpy HtoD]
                   24.34%  5.0240us         1  5.0240us  5.0240us  5.0240us  [CUDA memcpy DtoH]
                   16.74%  3.4560us         1  3.4560us  3.4560us  3.4560us  addInts(int*, int*, int)
      API calls:   99.34%  84.915ms         2  42.458ms  4.4530us  84.911ms  cudaMalloc
                    0.24%  208.04us         1  208.04us  208.04us  208.04us  cudaLaunchKernel
                    0.16%  137.74us       114  1.2080us     144ns  53.741us  cuDeviceGetAttribute
                    0.12%  101.00us         2  50.500us  7.0930us  93.908us  cudaFree
                    0.11%  89.781us         3  29.927us  22.937us  33.805us  cudaMemcpy
                    0.02%  13.175us    

# Implementing Embarassingly Parallel algorithms

yes that's a term, that's when threads don't need to interact at all. (very easy to make parallel). also called pleasingly parallel. We like embarassingly parallel.

https://www.youtube.com/watch?v=0ILeCeaor0A&list=PLKK11Ligqititws0ZOoGk3SW-TZCar4dK&index=6

Realistically you have some C++ code, then you want to parallelize it in CUDA. This takes a brute force of nearest neighbor and parallelizes it.




In [12]:
%%cuda
#include <iostream>
#include <ctime>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

using namespace std;

//brute force implementation on the CPU
void findClosestCPU(float3* points, int* indices, int count) {
    // Base case, if there's 1 point don't do anything
    if(count <=1) return;
    // Loop through every point
    for (int curPoint = 0; curPoint < count; curPoint++) {
        // This variable is nearest so far, set it to float. max
        float distToClosest = 3.4028238f ;
        // See how far itais from every other point
        for (int i = 0; i < count; i++) {
            // Don't check distance to itself
            if(i == curPoint) continue;
            float dist_sqr = (points[curPoint].x - points[i].x) *
                (points[curPoint].x - points[i].x) +
                (points[curPoint].y - points[i].y) *
                (points[curPoint].y - points[i].y) +
                (points[curPoint].z - points[i].z) *
                (points[curPoint].z - points[i].z);
            if(dist_sqr < distToClosest) {
                distToClosest = dist_sqr;
                indices[curPoint] = i;
            }
        }
    }
}

int main(){

    //defining parameters
    const int count = 10000;
    int* indexOfClosest = new int[count];
    float3* points = new float3[count];

    //defining random points
    for (int i = 0; i < count; i++){
        points[i].x = (float)(((rand()%10000))-5000);
        points[i].y = (float)(((rand()%10000))-5000);
        points[i].z = (float)(((rand()%10000))-5000);
    }

    long fastest = 1000000000;

    cout << "running brute force nearest neighbor on the CPU..."<<endl;
    for (int i = 0; i <= 10; i++){
        long start = clock();
        findClosestCPU(points, indexOfClosest, count);
        double duration = ( clock() - start ) / (double) CLOCKS_PER_SEC;
        cout << "test " << i << " took " << duration << " seconds" <<endl;
    }

    return 0;
}


running brute force nearest neighbor on the CPU...
test 0 took 1.17471 seconds
test 1 took 1.18766 seconds
test 2 took 1.18264 seconds
test 3 took 1.20396 seconds
test 4 took 1.18683 seconds
test 5 took 1.20544 seconds
test 6 took 1.19764 seconds
test 7 took 1.17379 seconds
test 8 took 1.19515 seconds
test 9 took 1.16731 seconds
test 10 took 1.17981 seconds



# Implementing on GPU
https://www.youtube.com/watch?v=UJ3kNwoxtuk&list=PLKK11Ligqititws0ZOoGk3SW-TZCar4dK&index=7


In [13]:
%%writefile findClosestGPU.cu
#include <iostream>
#include <ctime>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

using namespace std;

//this implementation is technically faster, but uses shared memory in an undefined
//way, and is thus fairly advanced.

//brute force implementation on the CPU
__global__ void findClosestGPU(float3* points, int* indices, int count) {
    if(count <= 1 ) return;
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < count){
        float3 thisPoint = points[idx];
        float smallestSoFar = 3.40282e38f;

        for (int i = 0; i < count; i++){
            if(i == idx) continue;

            float dist_sqr = (thisPoint.x - points [i].x) *
                (thisPoint.x - points[i].x) +
                (thisPoint.y - points[i].y) *
                (thisPoint.y - points[i].y) +
                (thisPoint.z - points[i].z) *
                (thisPoint.z - points[i].z);
            if(dist_sqr < smallestSoFar) {
                smallestSoFar = dist_sqr;
                indices[idx] = i;
            }
        }
    }
}

int main(){

    //defining parameters
    const int count = 10000;
    int* d_indexOfClosest = new int[count];
    float3* d_points = new float3[count];

    //defining random points
    for (int i = 0; i < count; i++){
        d_points[i].x = (float)(((rand()%10000))-5000);
        d_points[i].y = (float)(((rand()%10000))-5000);
        d_points[i].z = (float)(((rand()%10000))-5000);
    }

    int threads_per_block = 64;

    cout << "running brute force nearest neighbor on the CPU..."<<endl;
    for (int i = 1; i <= 10; i++){
        long start = clock();

        findClosestGPU<<<(count/threads_per_block)+1, threads_per_block>>>(d_points, d_indexOfClosest, count);
        cudaMemcpy(d_indexOfClosest, d_indexOfClosest, sizeof(int) * count, cudaMemcpyDeviceToHost);

        double duration = ( clock() - start ) / (double) CLOCKS_PER_SEC;
        cout << "test " << i << " took " << duration << " seconds" <<endl;
    }

    return 0;
}

Writing findClosestGPU.cu


In [14]:
!nvcc findClosestGPU.cu -o findClosestGPU.out
!./findClosestGPU.out

running brute force nearest neighbor on the CPU...
test 1 took 0.147381 seconds
test 2 took 8e-06 seconds
test 3 took 4e-06 seconds
test 4 took 4e-06 seconds
test 5 took 3e-06 seconds
test 6 took 3e-06 seconds
test 7 took 4e-06 seconds
test 8 took 4e-06 seconds
test 9 took 4e-06 seconds
test 10 took 3e-06 seconds


In [15]:
!nvprof ./findClosestGPU.out

running brute force nearest neighbor on the CPU...
==2430== NVPROF is profiling process 2430, command: ./findClosestGPU.out
test 1 took 0.102362 seconds
test 2 took 1.2e-05 seconds
test 3 took 8e-06 seconds
test 4 took 9e-06 seconds
test 5 took 7e-06 seconds
test 6 took 7e-06 seconds
test 7 took 6e-06 seconds
test 8 took 7e-06 seconds
test 9 took 7e-06 seconds
test 10 took 8e-06 seconds
==2430== Profiling application: ./findClosestGPU.out
==2430== Warning: 10 records have invalid timestamps due to insufficient device buffer space. You can configure the buffer space using the option --device-buffer-size.
==2430== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   99.75%  71.863ms        10  7.1863ms  4.5900us  71.812ms  cudaLaunchKernel
                    0.19%  137.44us       114  1.2050us     142ns  58.460us  cuDeviceGetAttribute
                    0.02%  14.921us        10  1.4920us     80

In [18]:
%%writefile findClosestGPU.cu
#include <iostream>
#include <ctime>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

using namespace std;

// Brute force implementation, parallelized on the GPU
__global__ void findClosestGPU(float3* points, int* indices, int count) {
    if (count <= 1) return;
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < count) {
        float3 thisPoint = points[idx];
        float smallestSoFar = 3.40282e38f;

        for (int i = 0; i < count; i++) {
            if (i == idx) continue;

            float dist_sqr = (thisPoint.x - points[i].x) *
                             (thisPoint.x - points[i].x) +
                             (thisPoint.y - points[i].y) *
                             (thisPoint.y - points[i].y) +
                             (thisPoint.z - points[i].z) *
                             (thisPoint.z - points[i].z);
            if (dist_sqr < smallestSoFar) {
                smallestSoFar = dist_sqr;
                indices[idx] = i;
            }
        }
    }
}

int main() {
    // Defining parameters
    const int count = 10000;
    int* h_indexOfClosest = new int[count];
    float3* h_points = new float3[count];

    // Defining random points
    for (int i = 0; i < count; i++) {
        h_points[i].x = (float)(((rand() % 10000)) - 5000);
        h_points[i].y = (float)(((rand() % 10000)) - 5000);
        h_points[i].z = (float)(((rand() % 10000)) - 5000);
    }

    // Device pointers
    int* d_indexOfClosest;
    float3* d_points;

    // Allocating memory on the device
    cudaMalloc(&d_indexOfClosest, sizeof(int) * count);
    cudaMalloc(&d_points, sizeof(float3) * count);

    // Copying values from the host to the device
    cudaMemcpy(d_points, h_points, sizeof(float3) * count, cudaMemcpyHostToDevice);

    int threads_per_block = 64;
    cout << "Running brute force nearest neighbor on the GPU..." << endl;
    for (int i = 1; i <= 10; i++) {
        long start = clock();

        findClosestGPU<<<(count / threads_per_block) + 1, threads_per_block>>>(d_points, d_indexOfClosest, count);
        cudaDeviceSynchronize();

        // Copying results from the device to the host
        cudaMemcpy(h_indexOfClosest, d_indexOfClosest, sizeof(int) * count, cudaMemcpyDeviceToHost);

        double duration = (clock() - start) / (double)CLOCKS_PER_SEC;
        cout << "Test " << i << " took " << duration << " seconds" << endl;
    }

    // Freeing device memory
    cudaFree(d_indexOfClosest);
    cudaFree(d_points);

    // Freeing host memory
    delete[] h_indexOfClosest;
    delete[] h_points;

    return 0;
}


Overwriting findClosestGPU.cu


In [19]:
!nvcc findClosestGPU.cu -o findClosestGPU.out
!./findClosestGPU.out

Running brute force nearest neighbor on the GPU...
Test 1 took 0.00265 seconds
Test 2 took 0.00242 seconds
Test 3 took 0.002423 seconds
Test 4 took 0.002416 seconds
Test 5 took 0.002414 seconds
Test 6 took 0.002412 seconds
Test 7 took 0.002409 seconds
Test 8 took 0.002411 seconds
Test 9 took 0.002412 seconds
Test 10 took 0.00241 seconds


In [20]:
!nvprof ./findClosestGPU.out

==2695== NVPROF is profiling process 2695, command: ./findClosestGPU.out
Running brute force nearest neighbor on the GPU...
Test 1 took 0.002646 seconds
Test 2 took 0.002424 seconds
Test 3 took 0.002424 seconds
Test 4 took 0.002422 seconds
Test 5 took 0.002418 seconds
Test 6 took 0.002423 seconds
Test 7 took 0.00245 seconds
Test 8 took 0.00242 seconds
Test 9 took 0.002416 seconds
Test 10 took 0.002421 seconds
==2695== Profiling application: ./findClosestGPU.out
==2695== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.76%  23.825ms        10  2.3825ms  2.3797ms  2.3857ms  findClosestGPU(float3*, int*, int)
                    0.19%  45.663us        10  4.5660us  4.4160us  5.0230us  [CUDA memcpy DtoH]
                    0.05%  12.800us         1  12.800us  12.800us  12.800us  [CUDA memcpy HtoD]
      API calls:   77.37%  84.653ms         2  42.326ms  7.5050us  84.645ms  cudaMalloc
                   21.82%  23.873